<a href="https://colab.research.google.com/github/devSeungGwan/novatus-academya/blob/master/Novatus_4_Titanic_%EC%83%9D%EC%A1%B4_%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Titanic 생존 문제

## 라이브러리 import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

## 데이터 import

In [ ]:
train_data = pd.read_csv('./titanic_train.csv')
test_data = pd.read_csv('./titanic_test.csv')

## 데이터 분석

In [ ]:
train_data.head(5)

In [ ]:
test_data.head(5)

In [ ]:
train_data.describe()

In [ ]:
train_data.describe(percentiles=[0.95])

In [ ]:
train_data.describe(include=['O'])

### 클래스 등급과 생존의 연관관계

In [ ]:
train_data[["Pclass", 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

> 클래스가 높을 수록 생존 확률이 높았다

### 성별에 따른 생존과 연관관계

In [ ]:
train_data[["Sex", 'Survived']].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

### 가족 관계에 따른 생존과 연관관계

In [ ]:
train_data[["SibSp", 'Survived']].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)

### 장소에 따른 생존과 연관관계

In [ ]:
train_data[["Embarked", 'Survived']].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)

> 연관관계가 그렇게 많진 않다

## 데이터 전처리

In [ ]:
prep_train_data = pd.read_csv('./titanic_train.csv')
prep_test_data = pd.read_csv('./titanic_test.csv')

In [ ]:
prep_train_data.columns

In [ ]:
prep_train_data.head(5)

In [ ]:
prep_train_data.drop(["Name", "Ticket", "Cabin" ], axis=1)

In [ ]:
def impute_age(cols):
    Age = cols[0]
    Pclass = cols[1]
    
    if pd.isnull(Age):
        if Pclass == 1: return 37
        elif Pclass == 2: return 29
        else: return 2
    else: return Age
    
def impute_fare(cols):
    Fare = cols[0]
    Pclass = cols[1]
    
    if pd.isnull(Fare):
        if Pclass == 1: return 92
        elif Pclass == 2: return 22
        else: return 12
    else: return Fare    
    
prep_train_data['Fare'] = prep_train_data[['Fare','Pclass']].apply(impute_fare,axis=1)
prep_train_data['Age'] = prep_train_data[['Age','Pclass']].apply(impute_age,axis=1)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
objList = prep_train_data.select_dtypes(include = "object").columns
print (objList)

for feat in objList:
    prep_train_data[feat] = le.fit_transform(prep_train_data[feat].astype(str))
print (prep_train_data.info())

In [ ]:
import seaborn as sns

sns.heatmap(prep_train_data.isnull(),yticklabels=False,cbar=False,cmap='viridis')

In [ ]:
X = prep_train_data.drop(['Survived', 'PassengerId'],axis=1) #features
y = prep_train_data['Survived'] #target

In [ ]:
X.head(5)

In [ ]:
y.head(5)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## 데이터 학습

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score

In [ ]:
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

In [ ]:
accuracy = classifier.score(X_train, y_train) * 100
y_train_pred = classifier.predict_proba(X_train)[:, 1]

In [ ]:
accuracy

In [ ]:
FPR, TPR, thresholds = roc_curve(y_train, y_train_pred)
AUC = roc_auc_score(y_train, y_train_pred)

plt.plot(FPR, TPR)
print("Accuracy: {}".format(accuracy))

In [ ]:
AUC